---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [11]:

import pandas as pd
import numpy as np
import datetime




def blight_model():
    
    # Your code here
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from adspy_shared_utilities import plot_decision_tree
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score
    
    # read files
    traind = pd.read_csv('train.csv', engine='python')
    test = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    # delete nan rows
    
    traind = traind[traind['compliance'].notnull()]
    
    # drop all not in the U.S
    traind = traind[traind.country == 'USA']
    test = test[test.country == 'USA']
    
    # merge latlons data
    
    traind = pd.merge(traind, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    
    traind = traind.set_index('ticket_id')
    test = test.set_index('ticket_id')
    
    # drop irrelevant data
    
    removelisttrain = ['hearing_date','non_us_str_code', 'clean_up_cost', 'inspector_name','violator_name',
                'violation_description', 'grafitti_status', 'admin_fee','state_fee',
                # leakingdata
                #'disposition', 'late_fee','discount_amount', 'judgment_amount',
                # columns not available in test
                'payment_amount', 'balance_due', 'payment_date', 'payment_status', 
                'collection_status', 'compliance_detail', 
                # address related columns
                'violation_zip_code', 'country', 'address', 'violation_street_number',
                'violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name', 
                'city', 'state', 'zip_code', 'address']
    
    removelisttest = ['hearing_date','non_us_str_code','clean_up_cost', 'inspector_name','violator_name',
                'violation_description', 'grafitti_status','admin_fee','state_fee',
                # leakingdata
                #'disposition', 'late_fee','discount_amount', 'judgment_amount',
                # address related columns
                'violation_zip_code', 'country', 'address', 'violation_street_number',
                'violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name', 
                'city', 'state', 'zip_code', 'address']
    traind.drop(removelisttrain, axis=1, inplace=True)
    test.drop(removelisttest, axis=1, inplace=True)
    
    # explore feature data
    
    #print(traind['agency_name'].value_counts(dropna=False))
    #print(test['agency_name'].value_counts(dropna=False))
    # print(traind['inspector_name'].value_counts(dropna=False))
    
    #print(traind['lat'].describe())
    #countnan = traind['lat'].isnull().sum()
    #print("lat nans: ", countnan)

    #print(traind['violation_code'].value_counts(dropna=False))
    #print(traind['disposition'].value_counts(dropna=False))
    #rint(traind['admin_fee'].value_counts(dropna=False))
    #rint(traind['state_fee'].value_counts(dropna=False))
    #print(traind['discount_amount'].value_counts(dropna=False))
    #print(traind['fine_amount'].describe())
    #print(traind['grafitti_status'].value_counts(dropna=False))
    #print(traind['judgment_amount'].describe())  
    
    #explore traind target data
    #print(traind['compliance'].value_counts(dropna=False))

    
    # feature manipulation

    # replace nans in lat lon
    traind['lat'] = traind['lat'].fillna(traind['lat'].mean())
    traind['lon'] = traind['lon'].fillna(traind['lon'].mean())
    test['lat'] = test['lat'].fillna(test['lat'].mean())
    test['lon'] = test['lon'].fillna(test['lon'].mean())
    
    # date to month of year
    traind['ticket_issued_date'] = pd.to_datetime(traind['ticket_issued_date'])
    traind['month'] = traind['ticket_issued_date'].dt.month
    traind['weekday'] = traind['ticket_issued_date'].dt.weekday
    #print(traind['month'].value_counts())
    #print(traind['weekday'].value_counts())
    traind.drop(['ticket_issued_date'],axis=1, inplace=True)
    
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
    test['month'] = test['ticket_issued_date'].dt.month
    test['weekday'] = test['ticket_issued_date'].dt.weekday
    #print(test['month'].value_counts())
    #print(test['weekday'].value_counts())
    test.drop(['ticket_issued_date'],axis=1, inplace=True)
    
     
    # delete single instance of neighborhood city halls and health department which is missing in test
    indexNames = traind[ traind['agency_name'] == "Neighborhood City Halls" ].index
    traind.drop(indexNames , inplace=True)
   
    indexNamest = test[ test['agency_name'] == "Neighborhood City Halls" ].index
    test.drop(indexNamest , inplace=True)
    
    indexNamesh = traind[ traind['agency_name'] == "Health Department" ].index
    traind.drop(indexNamesh , inplace=True)
    
    #agency name to dummy
    traind = pd.concat([traind,pd.get_dummies(traind['agency_name'], prefix='agency')],axis=1)
    traind.drop(['agency_name'],axis=1, inplace=True)
    
    test = pd.concat([test,pd.get_dummies(test['agency_name'], prefix='agency')],axis=1)
    test.drop(['agency_name'],axis=1, inplace=True)
    
    #disposition name to dummy
    traind = pd.concat([traind,pd.get_dummies(traind['disposition'], prefix='dis')],axis=1)
    traind.drop(['disposition'],axis=1, inplace=True)
    
    test = pd.concat([test,pd.get_dummies(test['disposition'], prefix='dis')],axis=1)
    test.drop(['disposition'],axis=1, inplace=True)
    
    #violation code to dummy
    violationc = traind['violation_code'].value_counts().index[15:]
    #print(violationc)
    traind['violation_code'] = traind['violation_code'].replace(violationc, np.nan)
    traind = pd.concat([traind,pd.get_dummies(traind['violation_code'], prefix='violation')],axis=1)
    traind.drop(['violation_code'],axis=1, inplace=True)

    test['violation_code'] = test['violation_code'].replace(violationc, np.nan)
    test = pd.concat([test,pd.get_dummies(test['violation_code'], prefix='violation')],axis=1)
    test.drop(['violation_code'],axis=1, inplace=True)
       
   
    

    #crosstabs
       
    ct2 = pd.crosstab(traind['month'],traind['compliance'])
    ct2_perc = ct2.divide(ct2.sum(axis=1), axis=0)
    #print(ct2_perc)
    #ct2_perc.plot.bar(stacked=True)
    
    ct3 = pd.crosstab(traind['weekday'],traind['compliance'])
    ct3_perc = ct3.divide(ct3.sum(axis=1), axis=0)
    #print(ct3_perc)
    #ct3_perc.plot.bar(stacked=True)
   
    ct4 = pd.crosstab(traind['agency_Buildings, Safety Engineering & Env Department'],traind['compliance'])
    ct4_perc = ct4.divide(ct4.sum(axis=1), axis=0)
    #print(ct4)
    
    

    
    #print final retained variables
    
    #print(traind.columns)
    #print(test.columns)    
    #print(traind.head())
    #print(test.head())
    #print(test.shape)

    # prep for split
    featurenames = ['fine_amount','late_fee', 'discount_amount',
       'judgment_amount', 'lat','month','agency_Buildings, Safety Engineering & Env Department','agency_Department of Public Works',
        'agency_Detroit Police Department', 'dis_Responsible (Fine Waived) by Deter',
       'dis_Responsible by Admission', 'dis_Responsible by Default',
       'dis_Responsible by Determination', 'violation_22-2-43', 'violation_22-2-45','violation_22-2-61','violation_9-1-103(C)',
        'violation_9-1-81(a)']
    y_tr = traind['compliance']
    X_tr = traind[featurenames]
    test = test[featurenames]
 

    #corrmatrix = traind.corr()
    #print(corrmatrix)
    
    #split and train
    X_train, X_test, y_train, y_test = train_test_split(X_tr, y_tr)
    
    #clf = DecisionTreeClassifier(max_depth= 4, random_state = 0).fit(X_train, y_train)
    #print('Accuracy of Decision Tree classifier on training set: {:.2f}'
    #    .format(clf.score(X_train, y_train)))
    #print('Accuracy of Decision Tree classifier on test set: {:.2f}'
    #     .format(clf.score(X_test, y_test)))
    
    reg_rf = RandomForestClassifier()
    grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
    grid_clf_auc = GridSearchCV(reg_rf, param_grid=grid_values, scoring='roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    
    print('Test set AUC: ', roc_auc_score(y_test, grid_clf_auc.predict_proba(X_test)[:,1]))
    print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    
    predict_y = grid_clf_auc.predict_proba(test)[:,1]
    test['compliance'] = predict_y
  
    return test.compliance

blight_model()

Test set AUC:  0.771360529267
Grid best parameter (max. AUC):  {'max_depth': 30, 'n_estimators': 100}
Grid best score (AUC):  0.760702484662


ticket_id
284932    0.000000
285362    0.000000
285361    0.010000
285338    0.260000
285346    0.040000
285345    0.570000
285347    0.190000
285342    0.990000
285530    0.050000
284989    0.000000
285344    0.040000
285343    0.000000
285340    0.000000
285341    0.200000
285349    0.030000
285348    0.610000
284991    0.000000
285532    0.100000
285406    0.000000
285001    0.000000
285006    0.000000
285405    0.000000
285337    0.000000
285496    0.000000
285497    0.000000
285378    0.000000
285589    0.000000
285585    0.000000
285501    0.000000
285581    0.000000
            ...   
376367    0.000000
376366    0.005590
376362    0.000000
376363    0.070000
376365    0.000000
376364    0.005590
376228    0.040964
376265    0.006491
376286    0.016265
376320    0.000995
376314    0.218108
376327    0.086056
376385    0.035482
376435    0.600000
376370    1.000000
376434    0.090000
376459    0.000000
376478    0.000000
376473    0.006809
376484    0.000000
376482    0.100000
37